In [1]:
import numpy as np
import scipy.stats
from scipy.stats import spearmanr
import scipy.spatial
from sklearn.model_selection import KFold
import random
from sklearn.metrics import mean_squared_error
from math import sqrt
import math
import warnings
import sys
import time
users = 1683
items = 1683
start=time.time();
#extract the data from file to matrix
f = open('u.data',"r")
data = []
for row in f:
    r = row.split('\t')
    e = [int(r[0]), int(r[1]), int(r[2])]
    data.append(e)
#print(data)
k_fold = KFold(n_splits=4) #to split and make a training set data 4folds
#print(k_fold.get_n_splits(data))
#print (k_fold.split(data))
Mat = np.zeros((users,items))
for e in data:
    Mat[e[0]-1][e[1]-1] = e[2]
    #print( Mat[e[0]-1][e[1]-1] )
item_similarity_pearson = np.zeros((items,items))
#print([item[0] for item in data])
#print (Mat[:][0])
print('start')
## find the similarity between two items using pearson coefficient
for item1 in range(items):
    #print (type(data))
    for item2 in range(items):
        #R = [item[item1] for item in data]
        #print('here', item1)
        #Q = np.array(data)[:,item2]
        if np.count_nonzero(Mat[:][item1]) and np.count_nonzero(Mat[:][item2]):
            #print('here')
            try:
                pearson_result = scipy.stats.pearsonr(Mat[:][item1],Mat[:][item2])
                #print(pearson_result,math.isnan(pearson_result[0]))
                if not math.isnan(pearson_result[0]):
                    item_similarity_pearson[item1][item2] = pearson_result[0]
                    #print(item_similarity_pearson[item1][item2])
                else:
                    item_similarity_pearson[item1][item2] = 0
            except:
                item_similarity_pearson[item1][item2] = 0
#print(item_similarity_pearson)
print('done')


start
done


In [3]:
sim_item_pearson = item_similarity_pearson
print(sim_item_pearson[9])
rmse = []
rmse_pearson = []
prec_on_top_k = []
spearman_pearson = []
precicion =0
for train_indices, test_indices in k_fold.split(data):
    match_prec = 0
    count_prec =0
    train = [data[i] for i in train_indices] #training data
    test = [data[i] for i in test_indices] #testing data 
    M = np.zeros((users,items))
    for e in train:
        M[e[0]-1][e[1]-1] = e[2]
    true_rate = []
    pred_rate_pearson = []
    
    #constructing baseline matrix
    baseline = np.zeros((users,items)) #user x item i mu + bx + bi
    avg_user = np.zeros(users)
    avg_item = np.zeros(items)
    total = 0.0
    #print('non', np.count_nonzero(M))
    ##for user mean rating
    for i in range(users):
        total = 0.0
        count = 0
        for j in range(items):
            if (M[int(i)][int(j)]>0):
                total = total + M[int(i)][int(j)]
                count = count+1
        if count>0:
            avg_user[int(i)] = total / count
    ##for item mean rating
    for i in range(items):
        total = 0.0
        count = 0
        for j in range(users):
            if (M[int(j)][int(i)]>0):
                total = total + M[int(j)][int(i)]
                count = count+1
        if(count>0):
            avg_user[int(i)] = total / float(count)
    ##calculate the mean of the matrix
    total_avg = 0.0
    count =0
    for i in range(users):
        for j in range(items):
            if(M[int(i)][int(j)]>0):
                total_avg = total_avg+M[int(i)][int(j)]
                count = count+1
    if(count>0):
        total_avg = total_avg / float(count)
        print(count)
    ##to calculate the baseline deviation values for each user and item combination
    for i in range(users):
        for j in range(items):
            baseline[int(i)][int(j)] = avg_user[int(i)] + avg_item[int(j)] - total_avg
    ##start predicting ratings
    for e in test:
            user = e[0]
            item = e[1]
            true_rate.append(e[2])
            pred_pearson = 3.0
            #print(np.count_nonzero(M[:][item-1]))
            if np.count_nonzero(M[:][item-1]):
                #print('here')
                sim_pearson = sim_item_pearson[:][item-1]
                ind = (M[user-1][:] > 0)
                #print(ind)
                #ind[item-1] = False
                ind2 = np.argpartition(sim_pearson, -10)[-10:]  #select the top 10 neighbors
                #print(np.count_nonzero(sim_pearson))
                #/print(ind2)
                #print('items ', items)
                for z in range(items):     ##make all the remaining values apart from the top10 as 0
                    #print(z)
                    if z not in ind2:
                        #print('in')
                        sim_pearson[int(z)] = 0  
                #print(np.count_nonzero(sim_pearson))
                normal_pearson = np.sum(np.absolute(sim_pearson[ind])) 
                if normal_pearson > 0:
                    tempo = np.zeros(items)
                    for temp in range(items):
                        tempo[int(temp)] = M[user-1][int(temp)] - baseline[user-1][int(temp)]                    
                    pred_pearson = baseline[user-1][item-1] + np.dot(sim_pearson,tempo)/normal_pearson #pearson similarity coefficient
                    #print(pred_pearson)
            if pred_pearson < 0:
                pred_pearson = 0
            if pred_pearson > 5:
                pred_pearson = 5
            print (str(user),"\t",str(item),"\t",str(e[2]),"\t", str(pred_pearson))
            pred_rate_pearson.append(pred_pearson)
            ##for precision
            count_prec = count_prec+1
            if (pred_pearson == e[2]):
                match_prec = match_prec +1
    precision = float((match_prec*100))/count_prec
    prec_on_top_k.append(precision)
    spcoef, p = spearmanr(true_rate, pred_rate_pearson) ##spearman test for evaluation
    spearman_pearson.append(spcoef)
    rmse_pearson.append(sqrt(mean_squared_error(true_rate, pred_rate_pearson))) ## root mean square
            #print (str(sqrt(mean_squared_error(true_rate, pred_rate_pearson))))
#print(rmse_pearson)
print("%s seconds" %(time.time()-start)) ##print the time taken. ##2580.360921372189 seconds
#print(np.mean(prec_on_top_k)) #79.85
print(np.mean(rmse_pearson))  #1.29621457813491894
#print(spearman_pearson)
print(np.mean(spearman_pearson))  #0.999985567943





[0. 0. 0. ... 0. 0. 0.]
75000
196 	 242 	 3 	 3.0
186 	 302 	 3 	 3.0
22 	 377 	 1 	 3.0
244 	 51 	 2 	 5
166 	 346 	 1 	 0
298 	 474 	 4 	 3.0
115 	 265 	 2 	 3.0
253 	 465 	 5 	 1.6797309072166413
305 	 451 	 3 	 2.968860955802504
6 	 86 	 3 	 4.321218958657724
62 	 257 	 2 	 3.0
286 	 1014 	 5 	 3.0
200 	 222 	 5 	 3.0
210 	 40 	 3 	 3.0
224 	 29 	 3 	 0.47988897966507676
303 	 785 	 3 	 2.937078381986717
122 	 387 	 5 	 3.0
194 	 274 	 2 	 2.167436736960892
291 	 1042 	 4 	 3.0
234 	 1184 	 2 	 3.0
119 	 392 	 4 	 0
167 	 486 	 4 	 5
299 	 144 	 4 	 5
291 	 118 	 2 	 3.606085540646944
308 	 1 	 4 	 5
95 	 546 	 2 	 0.829335574855555
38 	 95 	 5 	 3.0
102 	 768 	 2 	 3.0
63 	 277 	 4 	 3.0
160 	 234 	 5 	 4.48418704988587
50 	 246 	 3 	 0
301 	 98 	 4 	 5
225 	 193 	 4 	 3.0
290 	 88 	 4 	 3.0
97 	 194 	 3 	 3.0
157 	 274 	 4 	 3.0
181 	 1081 	 1 	 3.0
278 	 603 	 5 	 3.0
276 	 796 	 1 	 3.418345545521956
7 	 32 	 4 	 3.0
10 	 16 	 4 	 3.0
284 	 304 	 4 	 3.0
201 	 979 	 2 	 3.0
276

271 	 199 	 4 	 5
271 	 709 	 3 	 3.0
142 	 169 	 5 	 5
275 	 597 	 3 	 3.0
222 	 151 	 3 	 2.9995688305065875
87 	 40 	 3 	 0
207 	 258 	 4 	 0
272 	 1393 	 2 	 3.0
177 	 333 	 4 	 0.42044334884897494
207 	 1115 	 2 	 3.0
299 	 577 	 3 	 5
271 	 378 	 4 	 4.378675252357593
305 	 425 	 4 	 3.418879543762926
49 	 959 	 2 	 3.0
94 	 1224 	 3 	 3.0
130 	 1017 	 3 	 3.0
10 	 175 	 3 	 0
203 	 321 	 3 	 3.0
191 	 286 	 4 	 3.0
43 	 323 	 3 	 5
21 	 558 	 5 	 3.0
197 	 96 	 5 	 1.447225939163213
13 	 344 	 2 	 4.9845471199666616
194 	 66 	 3 	 1.7029177210624866
234 	 206 	 4 	 3.418954398083968
308 	 402 	 4 	 5
308 	 640 	 4 	 5
269 	 522 	 5 	 0
94 	 265 	 4 	 5
268 	 62 	 3 	 1.3821154932992918
272 	 12 	 5 	 0
121 	 291 	 3 	 3.6860727182805078
296 	 20 	 5 	 3.0
134 	 286 	 3 	 3.0
180 	 462 	 5 	 0
234 	 612 	 3 	 3.0
104 	 117 	 2 	 3.0
38 	 758 	 1 	 3.0
269 	 845 	 1 	 3.0
7 	 163 	 4 	 3.519659739051363
234 	 1451 	 3 	 3.0
275 	 405 	 2 	 3.0
52 	 250 	 3 	 4.050522052825182
102 

234 	 10 	 3 	 2.0742356212163098
280 	 1049 	 2 	 3.0
308 	 187 	 5 	 4.553034476326695
276 	 64 	 5 	 1.6976735337375142
192 	 948 	 3 	 3.0
122 	 509 	 4 	 3.0
85 	 588 	 3 	 5
262 	 931 	 2 	 3.0
201 	 272 	 3 	 3.0
181 	 870 	 2 	 3.0
295 	 739 	 4 	 5
263 	 568 	 4 	 3.0
295 	 39 	 4 	 5
201 	 1100 	 4 	 3.0
93 	 820 	 3 	 3.0
159 	 1028 	 5 	 3.0
158 	 665 	 2 	 3.0
293 	 423 	 3 	 1.1636852975377758
82 	 597 	 3 	 1.4337691971232236
276 	 181 	 5 	 3.364157971018268
13 	 823 	 5 	 4.934418136145245
217 	 2 	 3 	 3.0
83 	 660 	 4 	 3.0
189 	 20 	 5 	 0
222 	 796 	 4 	 3.0
146 	 1022 	 5 	 3.0
267 	 121 	 3 	 3.0
126 	 294 	 3 	 3.0
181 	 1060 	 1 	 3.0
125 	 80 	 4 	 2.6638695832231525
43 	 120 	 4 	 3.0
13 	 780 	 1 	 3.480116473850894
253 	 259 	 2 	 3.0
42 	 44 	 3 	 0.1319225424642759
77 	 518 	 4 	 3.0
291 	 686 	 5 	 2.831493672457186
268 	 21 	 3 	 0.31998311384082545
262 	 28 	 3 	 2.105943341976169
234 	 81 	 3 	 3.1210984025812074
29 	 245 	 3 	 3.0
236 	 57 	 5 	 3.0


186 	 148 	 4 	 3.0
312 	 692 	 4 	 3.0
91 	 683 	 3 	 3.0
5 	 454 	 1 	 3.3931029813211775
291 	 376 	 3 	 5
175 	 127 	 5 	 3.0
145 	 737 	 2 	 3.0
7 	 644 	 5 	 4.004569426971791
276 	 419 	 5 	 3.3335089461614307
83 	 210 	 5 	 3.0
102 	 524 	 3 	 5
153 	 174 	 1 	 3.0
62 	 302 	 3 	 3.0
49 	 995 	 3 	 3.0
268 	 298 	 3 	 3.0
207 	 554 	 2 	 3.0
313 	 616 	 5 	 0
286 	 44 	 3 	 1.6832941679849307
279 	 168 	 5 	 3.0
276 	 474 	 5 	 2.872364186207024
62 	 59 	 4 	 3.689570916754211
254 	 219 	 1 	 5
83 	 97 	 4 	 0
63 	 100 	 5 	 3.0
16 	 178 	 5 	 3.0
297 	 233 	 2 	 0
90 	 945 	 5 	 3.0
85 	 25 	 2 	 5
42 	 98 	 4 	 2.9837814691570226
303 	 393 	 4 	 3.0
274 	 50 	 5 	 3.0
104 	 299 	 3 	 3.0
94 	 792 	 4 	 5
184 	 98 	 4 	 4.312730136122996
293 	 708 	 3 	 2.639203503737861
248 	 589 	 4 	 3.0
18 	 950 	 3 	 3.0
217 	 27 	 1 	 4.818366404014585
200 	 892 	 4 	 3.0
201 	 148 	 1 	 1.952832811566679
296 	 222 	 5 	 5
7 	 662 	 3 	 0
196 	 381 	 4 	 3.0
69 	 427 	 3 	 3.0
72 	 196 	

106 	 582 	 4 	 3.0
181 	 1010 	 1 	 3.0
37 	 117 	 4 	 3.0
276 	 845 	 4 	 2.6357768743951793
22 	 258 	 5 	 3.0
70 	 82 	 4 	 2.309171665273096
5 	 98 	 3 	 5
308 	 95 	 4 	 5
60 	 208 	 5 	 3.0
270 	 778 	 5 	 3.331868066835657
243 	 208 	 4 	 3.0
92 	 540 	 2 	 3.0
81 	 280 	 4 	 3.0
293 	 412 	 1 	 3.2054835465021
200 	 478 	 5 	 2.3477083935967613
13 	 308 	 3 	 3.5045659162815475
56 	 184 	 4 	 3.0
116 	 250 	 4 	 0.023775686724762324
295 	 172 	 4 	 5
63 	 1007 	 5 	 3.0
295 	 235 	 4 	 3.0
104 	 1010 	 1 	 3.0
156 	 641 	 5 	 3.0
269 	 1165 	 1 	 3.0
160 	 430 	 5 	 3.0
237 	 191 	 4 	 3.0
287 	 252 	 1 	 3.0
290 	 132 	 3 	 3.0
45 	 109 	 5 	 3.0
224 	 678 	 3 	 3.0
145 	 764 	 2 	 3.0
277 	 1011 	 3 	 3.0
65 	 100 	 3 	 3.0
272 	 1101 	 5 	 3.0
116 	 255 	 3 	 3.0
184 	 86 	 5 	 3.0
285 	 151 	 5 	 3.0
222 	 148 	 2 	 2.498938238678173
72 	 28 	 4 	 3.0
271 	 187 	 5 	 3.0
94 	 211 	 5 	 5
246 	 425 	 5 	 3.0
115 	 8 	 5 	 3.0
176 	 327 	 3 	 3.0
13 	 396 	 3 	 2.23539713244

...

398 	 582 	 2 	 5
870 	 318 	 5 	 5
889 	 980 	 4 	 3.0
804 	 288 	 1 	 5
889 	 657 	 4 	 5
868 	 186 	 2 	 3.0
13 	 834 	 1 	 3.967458860151379
653 	 187 	 4 	 3.0
920 	 311 	 3 	 3.0
612 	 480 	 4 	 3.0
687 	 269 	 4 	 3.0
854 	 20 	 2 	 3.0
573 	 528 	 4 	 5
868 	 174 	 5 	 3.0
655 	 4 	 2 	 0.7439346333369838
853 	 682 	 4 	 0
251 	 25 	 4 	 0.6562319995824029
885 	 739 	 4 	 5
839 	 846 	 2 	 3.0
811 	 243 	 3 	 3.0
406 	 654 	 4 	 5
500 	 383 	 3 	 3.0
768 	 535 	 3 	 5
727 	 392 	 4 	 3.0
756 	 383 	 3 	 5
836 	 185 	 5 	 3.0
928 	 191 	 5 	 3.0
727 	 465 	 2 	 3.4111750510069974
401 	 243 	 3 	 5
666 	 483 	 5 	 5
896 	 566 	 4 	 3.226946334664058
472 	 96 	 5 	 5
361 	 1152 	 2 	 3.0
735 	 325 	 1 	 3.0
938 	 458 	 4 	 3.0
916 	 546 	 2 	 4.096670523545833
505 	 182 	 1 	 0
922 	 276 	 3 	 0.5522363651548405
450 	 178 	 4 	 5
179 	 313 	 4 	 3.0
923 	 866 	 4 	 3.0
639 	 1101 	 3 	 3.0
665 	 417 	 3 	 5
916 	 1073 	 4 	 3.0
61 	 301 	 1 	 3.0
846 	 435 	 5 	 5
606 	 87 	 4 	 2

698 	 515 	 4 	 3.0
22 	 68 	 4 	 0
334 	 99 	 4 	 3.9057966828034263
749 	 186 	 4 	 5
721 	 229 	 5 	 5
615 	 708 	 2 	 2.711024884204002
519 	 348 	 5 	 3.0
816 	 1025 	 4 	 3.0
466 	 161 	 2 	 2.820566655706443
915 	 258 	 2 	 3.0
666 	 91 	 3 	 3.0
916 	 498 	 3 	 3.0
940 	 1137 	 3 	 3.0
286 	 72 	 4 	 3.0
804 	 436 	 5 	 5
724 	 352 	 1 	 2.070425874458668
500 	 546 	 4 	 1.196415228642121
936 	 919 	 5 	 0
660 	 946 	 2 	 3.0
943 	 139 	 1 	 3.0
815 	 484 	 4 	 3.0
548 	 288 	 3 	 5
847 	 88 	 2 	 2.1955717741537844
303 	 546 	 2 	 3.0853308496287526
409 	 338 	 3 	 4.6919108203140105
355 	 319 	 5 	 3.0
774 	 453 	 2 	 5
501 	 274 	 3 	 4.078858234484699
790 	 153 	 3 	 5
535 	 638 	 4 	 5
913 	 462 	 3 	 3.0
712 	 400 	 3 	 5
488 	 633 	 5 	 0
761 	 245 	 5 	 3.0
782 	 1610 	 1 	 3.0
871 	 347 	 5 	 3.0
521 	 300 	 3 	 3.0
916 	 939 	 3 	 3.0
907 	 326 	 5 	 3.0
854 	 174 	 3 	 3.0
896 	 210 	 4 	 0.32431373668100294
838 	 283 	 5 	 3.0
281 	 748 	 5 	 5
938 	 685 	 3 	 5
286

816 	 690 	 4 	 3.0
788 	 443 	 4 	 5
880 	 168 	 3 	 5
399 	 343 	 2 	 3.0
758 	 919 	 5 	 5
932 	 197 	 5 	 3.0
908 	 732 	 3 	 3.0
59 	 602 	 2 	 0
691 	 1172 	 5 	 3.0
549 	 151 	 3 	 3.0
721 	 995 	 3 	 3.0
928 	 135 	 4 	 3.0
828 	 640 	 2 	 3.0
766 	 499 	 3 	 3.0
303 	 1217 	 1 	 3.0
16 	 39 	 5 	 4.066617909831038
562 	 82 	 5 	 5
224 	 731 	 4 	 3.0
608 	 93 	 4 	 3.0
927 	 38 	 5 	 5
492 	 1098 	 4 	 3.0
717 	 260 	 1 	 3.0
919 	 1514 	 2 	 3.0
861 	 305 	 4 	 3.0
943 	 1047 	 2 	 3.0
99 	 348 	 4 	 3.9717893838978036
804 	 310 	 4 	 5
449 	 198 	 4 	 3.0
842 	 333 	 4 	 3.0
214 	 250 	 2 	 3.0
933 	 568 	 2 	 3.0
561 	 417 	 2 	 3.3621882542973545
810 	 873 	 3 	 3.0
637 	 117 	 2 	 3.0
361 	 367 	 3 	 2.728140160212627
831 	 690 	 4 	 3.0
937 	 116 	 4 	 3.0
234 	 835 	 3 	 2.6823509959535863
94 	 1140 	 2 	 3.0
405 	 1246 	 1 	 3.0
264 	 209 	 5 	 3.0
886 	 1217 	 4 	 3.0
615 	 332 	 2 	 0.0897951623610263
287 	 895 	 2 	 3.466568240500741
592 	 1079 	 1 	 3.0
932 	 210 	

344 	 302 	 5 	 3.683890729993772
608 	 92 	 3 	 1.0599906389640532
940 	 215 	 2 	 3.0
632 	 134 	 5 	 0.8181033669184927
899 	 180 	 3 	 3.0
294 	 405 	 4 	 5
698 	 86 	 2 	 5
782 	 271 	 2 	 5
666 	 960 	 4 	 3.0
588 	 91 	 5 	 3.0
891 	 740 	 5 	 3.0
417 	 302 	 3 	 5
160 	 127 	 5 	 5
293 	 127 	 5 	 3.1917324328267083
802 	 333 	 4 	 5
486 	 252 	 3 	 0
699 	 764 	 3 	 3.0
533 	 8 	 3 	 2.8914881316368004
911 	 420 	 4 	 3.0
760 	 98 	 3 	 3.0
804 	 294 	 5 	 4.9486316973968645
532 	 916 	 3 	 5
737 	 100 	 5 	 3.0
148 	 127 	 1 	 5
506 	 686 	 3 	 2.4134538026602197
57 	 121 	 4 	 0
732 	 269 	 5 	 3.0
643 	 655 	 4 	 3.0
708 	 1079 	 1 	 3.0
684 	 409 	 3 	 3.0
912 	 501 	 4 	 3.0
541 	 1412 	 1 	 3.0
886 	 217 	 2 	 5
734 	 50 	 4 	 3.0
932 	 606 	 4 	 3.0
922 	 596 	 4 	 3.0
892 	 188 	 5 	 3.0
715 	 106 	 2 	 3.0
347 	 427 	 4 	 2.544598837997537
501 	 245 	 3 	 3.0
933 	 1037 	 1 	 3.0
889 	 513 	 4 	 4.16792922135007
617 	 498 	 3 	 0.1327794377335111
782 	 342 	 2 	 3.0
1

749 	 823 	 3 	 5
806 	 90 	 4 	 3.0
884 	 463 	 5 	 3.0
707 	 708 	 3 	 3.0
535 	 1170 	 3 	 3.0
383 	 200 	 5 	 3.0
903 	 651 	 5 	 3.0
887 	 274 	 1 	 2.6944957761327406
484 	 210 	 5 	 0
715 	 778 	 2 	 3.0
879 	 111 	 4 	 3.0
710 	 79 	 4 	 3.0
711 	 48 	 4 	 0
669 	 479 	 5 	 3.0
684 	 248 	 3 	 3.0
407 	 7 	 4 	 5
889 	 8 	 3 	 3.0
619 	 578 	 4 	 3.0
485 	 752 	 3 	 3.0
393 	 613 	 4 	 4.115258153780455
853 	 333 	 4 	 3.0
887 	 1063 	 1 	 3.0
838 	 313 	 5 	 5
447 	 770 	 3 	 3.7634481479831887
527 	 659 	 4 	 1.4515974650414796
606 	 97 	 5 	 0.21564682321284062
630 	 815 	 3 	 3.0
806 	 209 	 3 	 3.839485148859418
867 	 748 	 4 	 3.0
693 	 1136 	 3 	 3.0
648 	 318 	 3 	 0.40635624176407886
875 	 964 	 4 	 3.0
749 	 636 	 4 	 5
490 	 109 	 5 	 3.0
441 	 282 	 4 	 3.0
851 	 1013 	 2 	 3.0
883 	 234 	 4 	 5
102 	 7 	 2 	 3.7397111317293694
896 	 96 	 5 	 3.0
935 	 117 	 4 	 3.0
358 	 1005 	 4 	 3.0
874 	 286 	 4 	 3.0
896 	 511 	 5 	 3.0
828 	 531 	 4 	 3.0
835 	 174 	 5 	 3.0
